# AI537 HW 3: Cat, Dog, and Monkey Detection with YOLOv8

Object detection refers to the task where given a picture, the goal is for a network to predict all the categories of objects included in the picture and their corresponding bounding boxes.

<div align=center>
<img src="https://user-images.githubusercontent.com/17425982/220232979-fffa480b-9ae6-4601-8af6-4116265dc650.png" alt="object detection" width="800"/>
</div>

In this project, we will train/test a YOLOv8 object detector on a small cat/dog/monkey dataset. The whole process consists of the following steps:

- [Installation and Setup](#installation-and-setup)
- [Dataset](#dataset)
- [Config](#config)
- [Training](#training)
- [Testing](#testing)

## Installation and Setup

Please refer to the `README_AI537.md` for detailed installation and environment setup instructions. 

After installation/setup is complete, the next two cells can be run as a sanity check to ensure certain parts of the environment are setup correctly.

In [ ]:
# check 
!python -V
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version
# Check GPU
!nvidia-smi

In [ ]:
# Check PyTorch version
import torch
print(torch.__version__)
print(torch.cuda.is_available())

## Dataset

The cat_dog_monkey dataset is a three-category dataset consisting of 438/51/31 images in the training/validation/testing splits. The original dataset is from 
[Monkey, Cat and Dog detection](https://www.kaggle.com/datasets/tarunbisht11/yolo-animal-detection-small).

<div align=center style="display: flex; flex-direction: column;">
    <figure>
        <img src="example_imgs/cats_and_monkeys_009.jpg" alt="Image 1" style="height: 200px; width: auto; margin-bottom: 5px;">
        <figcaption style="text-align: center;">Monkey and cat</figcaption>
    </figure>
    <figure>
        <img src="example_imgs/dogs_and_cats_000.jpg" alt="Image 2" style="height: 200px; width: auto; margin-bottom: 5px;">
        <figcaption style="text-align: center;">Dog and cat</figcaption>
    </figure>
</div>



We have done some additional processing to the dataset for generating bounding box annotations in the COCO format as well as generating several different annotation files to be used for training. Please download the processed dataset from Canvas, and place the downloaded ```cat_dog_monkey_dataset/``` directory into the ```data/``` directory of the ```mmyolo_AI537_2024Spring``` project.  

If done correctly, data should be structured as follows:
```text
- mmyolo_AI537_2024Spring/
    - data/
        - cat_dog_monkey_dataset/
            - train/
                - cats_001.jpg
                - cats_002.jpg
                ...
            - val/
                - cats_000.jpg
                - cats_007.jpg
                ...
            - train_cat_annotation_100.json
            - train_cat_annotation_50.json
            - train_cat_annotation_20.json
            - train_cat_annotation_5.json
            - train_cat_annotation_0.json
            - val.json
```

The contents of the dataset are:
- The `data/cat_dog_monkey_dataset/train/` and `data/cat_dog_monkey_dataset/val/` directories store all the images.
- The `data/cat_dog_monkey_dataset/train_cat_annotation_*.json` and `data/cat_dog_monkey_dataset/val.json` files store annotations in COCO format.

## Config
In this project, you will train several YOLOv8 models on training datasets with various amount of ground truth annotations for cats. For each of the ground truth annotation files/datasets, the goal is to find the best model performance across the three classes (cat, dog, and monkey) on the validation dataset. You will also be exploring how performance is impacted when you have different amounts of ground truth cat annotations to use for training a model in a supervised learning setting.

Completing this assignment will require you to train your model with the different ground truth annotation files and to try out different combinations of loss functions common to object detection. To help with this, we have provided you a config file called `ai537_yolov8_config.py` located in the `configs/yolov8/` directory. All changes with regards to model training will only need to be made in this file.

Comments have been added throughout the config file outlining the major settings you are expected to play around with for this assignment. Please also refer to the `README_AI537.md` for full details on what parts of the config are expected to be changed for this assignment.


## Training
You will be training your model on various different annotated datasets. Specifying which dataset and which losses (and their loss weights) are being used should all be done in our config file `configs/yolov8/ai537_yolov8_config.py` (refer to `README_AI537.md`).

Once you have setup the config file, you will simply run the command below to train your model. Training each model should be relatively fast. For example, training for 20 epochs on a NVIDIA RTX 4090 GPU takes less than two minutes.

At the end of training, the performance on the validation set should be printed as follows:

```text
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.677
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.863
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.758
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.191
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.581
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.738
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.414
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.749
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.781
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.340
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.716
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.824
```

The above properties are printed via the COCO API. According to the rules defined by COCO, the cat_dog_monkey dataset contains small, median and large-sized objects. For this assignment, __AP@50__ and __AP@75__ are the most important metrics to evaluate your model performance against. The results shown at the ending of training are the mean AP (the average across all classes). To get the per class AP, see the [Testing](#Testing) section.

To train your model, run the following:

In [ ]:
!python tools/train.py configs/yolov8/ai537_yolov8_config.py

### Saved outputs
Upon running the above training command, a `work_dirs/ai537_yolov8_config/` directory will be automatically generated:

<div align=center>
<img src="example_imgs/training_directory.png" alt="image"/>
</div>

In this directory, the model weights checkpoint file (e.g., epoch_20.pth) and the training config file (e.g., ai537_yolov8_config.py) for the most recent run will be saved. __After a run, you will most likely want to rename these two files as they will be overwritten the next time you run the train command.__

### Additional Note
Some warning messages may appear during training. These are not errors and will not impact the performance of your model. Any actual errors should result in your model not training from the start.



### Resume training after an iterruption

If you stop training, you can add `--resume` to the end of the training command and the program will automatically resume training with the latest weights file from `work_dirs`.

In [ ]:
!python tools/train.py configs/yolov8/ai537_yolov8_config.py --resume

### Training visualization

MMYOLO currently supports local, TensorBoard, WandB and other back-end visualization. The default is to use local visualization, and you can switch to WandB and other real-time visualization of various indicators in the training process.

For details about how to enable training visualization, please check `demo/15_minutes_object_detection.ipynb`

## Testing
To evaluate your trained model on the validation set and get per class AP metrics (AP@50 and AP@75), run the following command __(Note that $\{EPOCHNUM\}$ in this command needs to be replaced with the epoch number of your last/best training iteration)__:

In [ ]:
!python tools/test.py configs/yolov8/ai537_yolov8_config.py \
                      work_dirs/ai537_yolov8_config/epoch_{EPOCHNUM}.pth \
                      --cfg-options test_evaluator.classwise=True \
                      --show-dir show_results

Running the above test command should print out per class average precision (AP) metrics that looks something like:
```text
+----------+-------+--------+--------+-------+-------+-------+
| category | mAP   | mAP_50 | mAP_75 | mAP_s | mAP_m | mAP_l |
+----------+-------+--------+--------+-------+-------+-------+
| cat      | 0.572 | 0.785  | 0.571  | nan   | 0.438 | 0.6   |
| dog      | 0.666 | 0.858  | 0.804  | nan   | 0.708 | 0.661 |
| monkey   | 0.663 | 0.867  | 0.699  | 0.21  | 0.61  | 0.779 |
+----------+-------+--------+--------+-------+-------+-------+
```

In addition to AP metrics, result images will be automatically saved to the ```work_dirs/ai537_yolov8_config/{TIMESTAMP}/show_results``` directory. Below is one of the result images where the left image is the ground truth annotation and the right image is the predicted annotation by the model.

<div align=center>
<img src="example_imgs/cats_and_monkeys_000.jpg" alt="result_img"/>
</div>
